In [1]:
from random import shuffle
import glob, h5py, pickle, cv2, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
width = 128
height = 128
channels = 3

sub = 'est_twoAct'
filename = 'datasets/'+sub+'.hdf5'
trainpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/est_train_twoAct/*_proj*.png'
testpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/est_test_twoAct/*_proj*.png'

print('File to be created: ' + filename)

File to be created: datasets/est_twoAct.hdf5


In [2]:
train_files2 = sorted(glob.glob(trainpath))
test_files2 = sorted(glob.glob(testpath))
print(len(train_files2))
cnt = 1
ang_tol = [5, 10, 15, 20, 25, 30, 35, 40, 45]
acc1 = []

for t in ang_tol:
    correct = 0
    wrong = 0
    train_labels = []
    train_images = []
    test_labels = []
    test_images = []
    cnt = 0
    
    for i in range(len(train_files2)):
        fname = train_files2[i].split('/')[-1]
        if '\\' in fname:
            fname = fname.split('\\')[-1]
        class_idx = [m.end() for m in re.finditer('class', fname)]
        ang_idx = [m.end() for m in re.finditer('ang', fname)]
        underscore_idx = [m.start() for m in re.finditer('_', fname)]
        classes = []
        classes.append(int(fname[class_idx[0]: underscore_idx[1]]))
        classes.append(int(fname[class_idx[1]: underscore_idx[5]]))
#         classes.append(int(fname[class_idx[2]: underscore_idx[9]]))
        angs = []
        angs.append(int(fname[ang_idx[0]: underscore_idx[2]]))
        angs.append(int(fname[ang_idx[1]: underscore_idx[6]]))
#         angs.append(int(fname[ang_idx[2]: underscore_idx[10]]))
        proj_ang = int(fname[fname.find('proj')+4: -4])
        
        for idx, a in enumerate(angs):
            if abs(proj_ang - a) < t:
                correct += 1
                clas = classes[idx]
                
                if clas == 13:
                    clas = 8
                elif clas == 14:
                    clas = 9
                    
                train_labels.append(clas-1)
                img = cv2.imread(train_files2[i])
                img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
                clear_output(wait=True)
                print('Loading image '+str(cnt)+'/'+str(len(train_files2))+ ' ' + fname)
                train_images.append(img)  
                cnt += 1
                correct += 1
                break
            else:
                wrong += 1
    print(np.unique(train_labels))
    train_labels = to_categorical(np.array(train_labels))
    train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
    print(train_images.shape)
    print(train_labels.shape)
    
    for i in range(len(test_files2)):
        fname = test_files2[i].split('/')[-1]
        if '\\' in fname:
            fname = fname.split('\\')[-1]
        class_idx = [m.end() for m in re.finditer('class', fname)]
        ang_idx = [m.end() for m in re.finditer('ang', fname)]
        underscore_idx = [m.start() for m in re.finditer('_', fname)]
        classes = []
        classes.append(int(fname[class_idx[0]: underscore_idx[1]]))
        classes.append(int(fname[class_idx[1]: underscore_idx[5]]))
#         classes.append(int(fname[class_idx[2]: underscore_idx[9]]))
        angs = []
        angs.append(int(fname[ang_idx[0]: underscore_idx[2]]))
        angs.append(int(fname[ang_idx[1]: underscore_idx[6]]))
#         angs.append(int(fname[ang_idx[2]: underscore_idx[10]]))
        proj_ang = int(fname[fname.find('proj')+4: -4])
        
        for idx, a in enumerate(angs):
            if abs(proj_ang - a) < t:
                correct += 1
                clas = classes[idx]
                
                if clas == 13:
                    clas = 8
                elif clas == 14:
                    clas = 9
                    
                test_labels.append(clas-1)
                img = cv2.imread(test_files2[i])
                img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
                clear_output(wait=True)
                print('Loading image '+str(cnt)+'/'+str(len(test_files2))+ ' ' + fname)
                test_images.append(img)  
                cnt += 1
                correct += 1
                break
            else:
                wrong += 1
    print(np.unique(test_labels))
    test_labels = to_categorical(np.array(test_labels))
    test_images = np.swapaxes(np.array(test_images), 1, 2).reshape(len(test_images), width, height, channels)/255.
    print(test_images.shape)
    print(test_labels.shape)
    
    filename2 = filename[:-5] + '_angtol' + str(t) + '.hdf5'
    f = h5py.File(filename2, mode='w')
    f.create_dataset("train_img", train_images.shape, np.uint8)
    f.create_dataset("test_img", test_images.shape, np.uint8)  
    f.create_dataset("train_labels", train_labels.shape, np.uint8)
    f.create_dataset("test_labels", test_labels.shape, np.uint8)

    f["train_img"][...] = train_images
    f["test_img"][...] = test_images
    f["train_labels"][...] = train_labels
    f["test_labels"][...] = test_labels
    f.close()
    print(filename2 + ' created.')

Loading image 1663/531 pSpot_class4_ang0_iter8_pSpot_class3_ang-45_iter1_proj4.png
[0 1 2 3 4 5 6 7 8]
(526, 128, 128, 3)
(526, 9)
datasets/est_twoAct_angtol45.hdf5 created.
